In [ ]:
# default_exp samplers.datatypes

# Datatypes

> Collection of datatypes used by Sampler classes.

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export
from anytree import RenderTree, NodeMixin
import numpy as np
import qsam.math as math

In [ ]:
#export
class CountNode(NodeMixin):
    def __init__(self, name, counts=0, parent=None, children=None):
        super(CountNode, self).__init__()
        self.name = name
        self.counts = counts
        
        self.parent = parent
        if children:
            self.children = children
    
    def __str__(self):
        return f"{self.name} ({self.counts}/{self.counts})"
    
class Variable(CountNode):
        
    @property
    def rate(self):
        if self.parent.counts < 2: return 0.5
        else: return self.counts / self.parent.counts
    
    @property
    def var(self, var_fn=math.Wilson_var):
        if self.parent.counts < 1: return 1
        else: return var_fn(self.rate, self.parent.counts)
    
    def __str__(self):
        var = 0 if self.parent._deterministic else self.var
        return f"{self.name} ({self.counts}/{self.parent.counts}, {var:.2e})"
    
class Fail(Variable):
    def __init__(self, name, counts=0, parent=None, children=None):
        super(Fail, self).__init__(name, counts, parent, children)
        
class NoFail(Variable):
    def __init__(self, name, counts=0, parent=None, children=None):
        super(NoFail, self).__init__(name, counts, parent, children)
    
class Constant(CountNode):
    def __init__(self, name, data=dict(), deterministic=False, counts=0, parent=None, children=None):
        super(Constant, self).__init__(name, counts, parent, children)
        self.data = data
        self._deterministic = deterministic
    
    def __str__(self):
        out_str = []
        for k, v in self.data.items():
            if isinstance(v, (np.ndarray, list, tuple, set)):
                if len(v) == 1: elem_str = f'{v[0]:.2e}'
                elif len(v) == 2: elem_str = f'{v[0]:.2e},{v[-1]:.2e}'
                else: elem_str = f'{v[0]:.2e},...,{v[-1]:.2e}'
            else:
                elem_str = f'{v:.2e}'
            out_str.append( f'{k}: {elem_str}' )
        return f"{self.name} " + "(" + ", ".join(out_str) + ")"
    
class Tree:
    def __init__(self, root=None):
        self.root = root
        
    def add(self, name, node_cls, parent=None, **kwargs):
        if parent == None:
            if self.root == None: 
                self.root = node_cls(name, **kwargs)
            return self.root
        else:
            child_match = [n for n in parent.children if n.name == name]
            if child_match:
                return child_match[0]
            else:
                return node_cls(name, parent=parent, **kwargs)
        
    def detach(self, node):
        node.parent = None
        
    def rate(self, ckey):
        p = 0
        for leaf in self.root.leaves:
            if not leaf.is_root and not leaf.parent._deterministic: 
                if isinstance(leaf, NoFail): 
                    continue
                prod = 1
                for node in leaf.path[1:]:
                    if isinstance(node, Constant): prod *= node.data[ckey]
                    elif isinstance(node, Variable): prod *= node.rate
                p += prod
        return p
    
    def var(self, ckey):
        var = 0
        for node in [n for n in self.root.descendants if isinstance(n, Variable)]:
            if not node.parent._deterministic:# and not node.name == 'END':
                twig = 1
                for n in node.path[1:-1]:
                    if isinstance(n, Constant): twig *= n.data[ckey]
                    elif isinstance(n, Variable): twig *= n.rate
                
                subtree = 0
                for leaf in node.leaves:
                    if isinstance(leaf, NoFail): 
                        continue
                    prod = 1
                    for n in leaf.iter_path_reverse():
                        if n == node: break
                        elif isinstance(n, Constant): prod *= n.data[ckey]
                        elif isinstance(n, (Variable,Fail)): prod *= n.rate
                    subtree += prod
                var += node.var * twig**2 * subtree**2
        return var
    
    def delta(self, ckey):
        path_sum = 0.0
        for leaf in self.root.leaves:
            if not leaf.is_root:
                prod = 1.0
                for node in leaf.path[1:]:
                    if isinstance(node, Constant): prod *= node.data[ckey]
                    elif isinstance(node, (Variable,Fail,NoFail)): prod *= node.rate
                path_sum += prod
        return 1.0 - path_sum
    
    def __str__(self):
        return '\n'.join([f'{pre}{node}' for pre, _, node in RenderTree(self.root)])